# Tensorflow 2 quickstart for Experts

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
print(tf.__version__)

2.4.0


In [31]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train, X_test = X_train / 255.0, X_test / 255.0

In [32]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)


In [33]:
X_train = X_train[..., tf.newaxis].astype("float32")
X_test = X_test[..., tf.newaxis].astype("float32")

In [34]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(60000, 28, 28, 1)
(60000,)
(10000, 28, 28, 1)
(10000,)


In [40]:
dataset = tf.data.Dataset.from_tensor_slices(np.arange(0, 9).reshape(3, 3))
for element in dataset:
    print(type(element))
    print(element)

<class 'tensorflow.python.framework.ops.EagerTensor'>
tf.Tensor([0 1 2], shape=(3,), dtype=int64)
<class 'tensorflow.python.framework.ops.EagerTensor'>
tf.Tensor([3 4 5], shape=(3,), dtype=int64)
<class 'tensorflow.python.framework.ops.EagerTensor'>
tf.Tensor([6 7 8], shape=(3,), dtype=int64)


In [41]:
train_ds = tf.data.Dataset.from_tensor_slices(
    (X_train, y_train)).shuffle(10000).batch(32)

In [42]:
test_ds = tf.data.Dataset.from_tensor_slices(
    (X_train, y_train)).batch(32)

## Model Definition

In [43]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, Flatten, Dense

class MyModel(Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = Conv2D(32, 3, activation='relu')
        self.flatten = Flatten()
        self.d1 = Dense(128, activation='relu')
        self.d2 = Dense(10)

    def call(self, x):
        x = self.conv1(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)


In [44]:
model = MyModel()

In [45]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [46]:
optimizer = tf.keras.optimizers.Adam()

In [47]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

## Train the model

In [48]:
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = model(images, training=True)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)

## Test the model

In [49]:
@tf.function
def test_step(images, labels):
    predictions = model(images, training=False)
    loss = loss_object(labels, predictions)

    test_loss(loss)
    test_accuracy(labels, predictions)

In [50]:
EPOCHS = 10

for epoch in range(EPOCHS):
    train_loss.reset_states()
    test_loss.reset_states()
    train_accuracy.reset_states()
    test_accuracy.reset_states()

    for images, labels in train_ds:
        train_step(images, labels)
    
    for test_images, test_labels in test_ds:
        test_step(test_images, test_labels)

    print(
        f'Epoch {epoch + 1}, '
        f'Loss: {train_loss.result()}, '
        f'Accuracy: {train_accuracy.result() * 100}, '
        f'Test Loss: {test_loss.result()}, '
        f'Test Accuracy: {test_accuracy.result() * 100}'
    )

Epoch 1, Loss: 0.1330946832895279, Accuracy: 95.96666717529297, Test Loss: 0.04031111299991608, Test Accuracy: 98.83833312988281
Epoch 2, Loss: 0.041578397154808044, Accuracy: 98.68999481201172, Test Loss: 0.0215348731726408, Test Accuracy: 99.34832763671875
Epoch 3, Loss: 0.022517668083310127, Accuracy: 99.2683334350586, Test Loss: 0.012359843589365482, Test Accuracy: 99.61166381835938
Epoch 4, Loss: 0.013063706457614899, Accuracy: 99.57833099365234, Test Loss: 0.008732064627110958, Test Accuracy: 99.72166442871094
Epoch 5, Loss: 0.008551204577088356, Accuracy: 99.71666717529297, Test Loss: 0.004480674862861633, Test Accuracy: 99.86499786376953
Epoch 6, Loss: 0.0076472884975373745, Accuracy: 99.7266616821289, Test Loss: 0.007898394949734211, Test Accuracy: 99.74832916259766
Epoch 7, Loss: 0.0046086544170975685, Accuracy: 99.85832977294922, Test Loss: 0.0013280868297442794, Test Accuracy: 99.96333312988281
Epoch 8, Loss: 0.004008762072771788, Accuracy: 99.85166931152344, Test Loss: 0.0